In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import asyncio
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
from typing import Literal
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = '{:.4f}'.format

In [3]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [4]:
async def get_expiry(f_today):
    
    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24 :
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(start_date=nearest_wednesday, end_date=nearest_wednesday).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_nifty(f_today):
    
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [14]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
bnf_1min = pd.read_csv('../data/bnf_min.csv')
# bnf_1min = pd.read_csv('../data/nifty_min.csv')
bnf_1min['datetime'] = pd.to_datetime(bnf_1min['datetime'])
bnf_1min = bnf_1min[bnf_1min['datetime'].dt.year >= 2024]

In [15]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [16]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [17]:
# bnf_1min

In [18]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label='left',
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [29]:
def calculate_weekly_ranges(df):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # Resample to weekly data to get the high and low for each week
    weekly_ranges = df.resample('W-FRI').agg({'high': 'max', 'low': 'min'})
    return weekly_ranges

async def find_breakouts(df, weekly_ranges):
    
    results = []
    multiplier_to_range = 0.2

    portfolio_value = 10_00_000
    index_leverage = 6.5

    index_name = 'BANKNIFTY'
    index_str_for_opt = 'bnf'
    
    weekly_ranges['datetime'] = pd.to_datetime(weekly_ranges['datetime'])
    weekly_ranges = weekly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    
    for i in range(1, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i-1]
        current_week = weekly_ranges.iloc[i]
        print('Current Week : ', current_week['datetime'].date())
        if current_week['datetime'].date() in dates_to_avoid:
            print('Date Avoided')
            continue
        previous_week_high = weekly_ranges.iloc[i-1]['high']
        previous_week_low = weekly_ranges.iloc[i-1]['low']

        current_week_open = weekly_ranges.iloc[i]['open']
        
        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[(df['datetime'] >= current_week['datetime']) & (df['datetime'] <= (current_week['datetime'] + pd.Timedelta(days=6)))]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data['open'].iloc[0] + addition_range
        low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)
        
        breakout_high = current_week_data[current_week_data['high'] >= high_level]
        breakout_low = current_week_data[current_week_data['low'] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]['datetime']
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(exit_date)
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type='PE',
            )
            pe_df = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=exit_date,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class='P',
            )
            if not isinstance(pe_df, str):
                pe_df = pe_df.to_pandas()            
                entry_price = pe_df.iloc[0]['c']
            else:
                entry_price = float('nan')
            
            # Exit Scenario
            current_week_data_after_entry = current_week_data[(current_week_data['datetime'] >= breakout_high_time) & (current_week_data['datetime'].dt.date <= exit_date - dt.timedelta(days=0))]
            # print(current_week_data_after_entry.head())
            low_breach = current_week_data_after_entry[current_week_data_after_entry['low'] <= previous_week_low]
            # print(len(low_breach))
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]['datetime']
                # print(exit_time)
                remark = 'SL Hit'
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    exit_price = pe_df.loc[(pe_df['datetime'].dt.date >= exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
                    
                else:
                    exit_price = float('nan')
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(exit_date - dt.timedelta(days=1), dt.time(15, 20))
                remark = 'Friday Closing'
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]['c']
                else:
                    exit_price = float('nan')
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = (portfolio_value * index_leverage / high_level)
            pnl = qty * (entry_price - exit_price)
            unit = {
                'Week': previous_week['datetime'].date(),
                'Week High': previous_week_high,
                'Week Low': previous_week_low,
                'Weekly Range': weekly_range,
                'Current Week Open': current_week_open,
                'Multiplier to Range': f'{int(multiplier_to_range * 100)}%',
                'Entry Level Long': high_level,
                'Entry Level Short': low_level,
                'Break Type': 'high', 
                'Strike': atm_strike,
                'Expiry': expiry,
                'DTE': dte,
                'Contract': contract,
                'Option Type': contract[-2:],
                'Entry Time': breakout_high_time,
                'Entry Price': entry_price,
                'Exit Time': exit_time,
                'Exit Price': exit_price,
                'Remark': remark,
                'Points': entry_price - exit_price,
                'Qty': qty,
                'PnL': pnl,
                'ROI%': (pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]['datetime']
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(exit_date)
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type='CE',
            )
            ce_df = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=exit_date,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class='C',
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str):
                ce_df = ce_df.to_pandas()            
                entry_price = ce_df.iloc[0]['c']
            else:
                entry_price = float('nan')

            # Exit Scenario
            current_week_data_after_entry = current_week_data[(current_week_data['datetime'] >= breakout_low_time) & (current_week_data['datetime'].dt.date <= exit_date - dt.timedelta(days=0))]
            # print(current_week_data_after_entry.head())
            high_breach = current_week_data_after_entry[current_week_data_after_entry['high'] >= previous_week_high]
            # print(len(high_breach))
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]['datetime']
                # print(exit_time)
                remark = 'SL Hit'
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    exit_price = ce_df.loc[(ce_df['datetime'].dt.date >= exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
                else:
                    exit_price = float('nan')
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(exit_date - dt.timedelta(days=1), dt.time(15, 20))
                remark = 'Friday Closing'
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]['c']
                else:
                    exit_price = float('nan')
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            
            qty = (portfolio_value * index_leverage / high_level)
            pnl = qty * (entry_price - exit_price)
            unit = {
                'Week': previous_week['datetime'].date(),
                'Week High': previous_week_high,
                'Week Low': previous_week_low,
                'Weekly Range': weekly_range,
                'Current Week Open': current_week_open,
                'Multiplier to Range': f'{int(multiplier_to_range * 100)}%',
                'Entry Level Long': high_level,
                'Entry Level Short': low_level,
                'Break Type': 'low', 
                'Strike': atm_strike,
                'Expiry': expiry,
                'DTE': dte,
                'Contract': contract,
                'Option Type': contract[-2:],
                'Entry Time': breakout_low_time,
                'Entry Price': entry_price,
                'Exit Time': exit_time,
                'Exit Price': exit_price,
                'Remark': remark,
                'Points': entry_price - exit_price,
                'Qty': qty,
                'PnL': pnl,
                'ROI%': (pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

async def trade():
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    weekly_ranges = resample(pl.DataFrame(df), '7d', pd.Timedelta(days=4))
    weekly_ranges = weekly_ranges.to_pandas()
    
    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas

tradebook = await trade()
# tradebook

Current Week :  2017-01-09
Current Week :  2017-01-16
Current Week :  2017-01-23
Current Week :  2017-01-30
Current Week :  2017-02-06
Current Week :  2017-02-13
Current Week :  2017-02-20
Current Week :  2017-02-27
Current Week :  2017-03-06
Current Week :  2017-03-13
Current Week :  2017-03-20
Current Week :  2017-03-27
Current Week :  2017-04-03
Current Week :  2017-04-10
Current Week :  2017-04-17
Current Week :  2017-04-24
Current Week :  2017-05-01
Current Week :  2017-05-08
Current Week :  2017-05-15
Current Week :  2017-05-22
Current Week :  2017-05-29
Current Week :  2017-06-05
Current Week :  2017-06-12
Current Week :  2017-06-19
Current Week :  2017-06-26
Current Week :  2017-07-03
Current Week :  2017-07-10
Current Week :  2017-07-17
Current Week :  2017-07-24
Current Week :  2017-07-31
Current Week :  2017-08-07
Current Week :  2017-08-14
Current Week :  2017-08-21
Current Week :  2017-08-28
Current Week :  2017-09-04
Current Week :  2017-09-11
Current Week :  2017-09-18
C

In [30]:
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Qty,PnL,ROI%
0,2017-01-02,18323.0000,17831.7500,491.2500,18314.2500,20%,18412.5000,18216.0000,high,18200,2017-01-19,9,BANKNIFTY1711918200PE,PE,2017-01-10 15:17:00,93.4000,2017-01-13 15:20:00,7.6000,Friday Closing,85.8000,353.0210,30289.2057,3.0289
1,2017-01-09,18965.7000,18257.7500,707.9500,18899.7000,20%,19041.2900,18758.1100,high,18800,2017-01-25,9,BANKNIFTY17JAN18800PE,PE,2017-01-16 12:54:00,88.8500,2017-01-20 15:20:00,107.3500,Friday Closing,-18.5000,341.3634,-6315.2234,-0.6315
2,2017-01-16,19276.2500,18793.3500,482.9000,18762.0000,20%,18858.5800,18665.4200,high,18700,2017-02-02,10,BANKNIFTY1720218700PE,PE,2017-01-23 09:46:00,174.2500,2017-01-23 12:18:00,177.2000,SL Hit,-2.9500,344.6707,-1016.7786,-0.1017
3,2017-01-23,19794.9000,18725.0500,1069.8500,19718.8000,20%,19932.7700,19504.8300,high,19500,2017-02-09,8,BANKNIFTY1720919500PE,PE,2017-02-01 13:06:00,100.0000,2017-02-03 15:20:00,12.8000,Friday Closing,87.2000,326.0962,28435.5862,2.8436
4,2017-01-23,19794.9000,18725.0500,1069.8500,19718.8000,20%,19932.7700,19504.8300,low,19900,2017-02-09,9,BANKNIFTY1720919900CE,CE,2017-01-31 09:25:00,164.0000,2017-02-01 13:02:00,197.0500,SL Hit,-33.0500,326.0962,-10777.4785,-1.0777
5,2017-01-30,20228.9500,19435.7500,793.2000,20307.1500,20%,20465.7900,20148.5100,low,20500,2017-02-16,8,BANKNIFTY1721620500CE,CE,2017-02-08 14:30:00,83.0000,2017-02-08 14:45:00,97.8000,SL Hit,-14.8000,317.6032,-4700.5271,-0.4701
6,2017-02-06,20461.3000,20001.1000,460.2000,20273.6500,20%,20365.6900,20181.6100,high,20200,2017-02-23,8,BANKNIFTY17FEB20200PE,PE,2017-02-15 09:39:00,108.0000,2017-02-17 15:20:00,43.0000,Friday Closing,65.0000,319.1642,20745.6757,2.0746
7,2017-02-06,20461.3000,20001.1000,460.2000,20273.6500,20%,20365.6900,20181.6100,low,20400,2017-02-23,10,BANKNIFTY17FEB20400CE,CE,2017-02-13 12:15:00,127.7000,2017-02-17 09:15:00,299.6500,SL Hit,-171.9500,319.1642,-54880.2913,-5.4880
8,2017-02-13,21031.6500,20088.4000,943.2500,20494.8500,20%,20683.5000,20306.2000,high,20300,2017-03-02,10,BANKNIFTY1730220300PE,PE,2017-02-20 15:16:00,78.0000,2017-02-24 15:20:00,18.5000,Friday Closing,59.5000,314.2602,18698.4795,1.8698
9,2017-02-20,21011.9000,20474.8500,537.0500,20853.4500,20%,20960.8600,20746.0400,low,21000,2017-03-09,10,BANKNIFTY1730921000CE,CE,2017-02-27 09:35:00,119.9000,2017-03-03 15:20:00,20.3500,Friday Closing,99.5500,310.1018,30870.6322,3.0871


In [31]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook['Points'].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

20336.350000000002

In [32]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook['Entry Time'] = pd.to_datetime(tradebook['Entry Time'])
tradebook['Trade Year'] = tradebook['Entry Time'].dt.year

In [33]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI%'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['ROI%'] > 0]['ROI%'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['ROI%'] < 0]['ROI%'].mean()
overall_max_drawdown = (combined_df_sorted['ROI%'].cumsum() - combined_df_sorted['ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,27.3276,70,54.2857,2.0820,-1.7858,-9.8888,2.7635
2018,36.4082,73,57.5342,2.4776,-2.4161,-12.2698,2.9673
2019,41.6894,80,58.7500,2.8914,-2.9439,-12.6905,3.2851
2020,211.2963,66,77.2727,5.1379,-3.6239,-14.8500,14.2287
2021,48.1458,74,56.7568,4.0403,-4.3410,-19.9859,2.4090
2022,79.9408,81,54.3210,3.8836,-2.7557,-11.5283,6.9343
2023,2.2602,75,49.3333,1.8120,-2.6993,-16.7770,0.1347
2024,14.3754,17,76.4706,2.1422,-3.3684,-8.2268,1.7474
Overall,461.4436,536.0000,58.5821,3.2375,-2.8913,-19.9859,23.0884


In [34]:
stats_of_trades = tradebook
stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
grouped

,Option Type,ROI%,Max Drawdown,ROI/DD Ratio
0,CE,207.6905,-23.8836,8.6959
1,PE,253.7531,-31.0431,8.1742


In [35]:
# df.to_csv('JJMS Weekly Opt Selling var3 20pct w cs.csv')

In [19]:
df = tradebook

In [20]:
df['Slippage in pts'] = df.apply(lambda row: (row['Entry Price'] + row['Exit Price']) * 0.01, axis=1)
df['PnL w cs'] = df.apply(lambda row: (row['PnL'] - (row['Slippage in pts'] * row['Qty'])), axis=1)

In [30]:
# df

In [25]:
df['ROI% w cs'] = df.apply(lambda row: (row['PnL w cs'] * 100/ 1000000), axis=1)

In [27]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = df
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI% w cs'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI% w cs'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI% w cs'] > 0]['ROI% w cs'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI% w cs'] < 0]['ROI% w cs'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI% w cs'].cumsum() - year_trades['ROI% w cs'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI% w cs'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['ROI% w cs'] > 0]['ROI% w cs'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['ROI% w cs'] < 0]['ROI% w cs'].mean()
overall_max_drawdown = (combined_df_sorted['ROI% w cs'].cumsum() - combined_df_sorted['ROI% w cs'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,21.3990,63,57.1429,1.8500,-1.8080,-8.6980,2.4602
2018,28.8369,73,57.5342,2.2431,-2.2542,-12.1399,2.3754
2019,41.2181,75,61.3333,2.5575,-2.6354,-12.1219,3.4003
2020,177.3658,63,77.7778,4.4069,-2.7552,-14.0652,12.6102
2021,36.5509,74,56.7568,3.6554,-3.6555,-18.9489,1.9289
2022,65.7697,81,54.3210,3.5166,-2.4043,-11.5208,5.7088
2023,-1.6821,75,49.3333,1.6342,-2.2195,-16.0807,-0.1046
2024,11.6885,17,76.4706,1.9288,-3.3464,-8.4881,1.3770
Overall,381.1468,521.0000,59.3090,2.8744,-2.5608,-18.9489,20.1144


Stats for Variation 20% movement , Sell Strike at Range level, Exit at Prev Wk High/Low with SLIPPAGES !

In [18]:
# tradebook

In [ ]:
def round_to_nearest_100(value):
    return 100 * round(value / 100)
    
async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 1
    index_name = 'BANKNIFTY'
    index_str_for_opt = 'bnf'
    
    df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    
    df['Hedge Contract'] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, 
            row['Hedge Strike'], 
            row['Expiry'], 
            row['Contract'][-2:]
        ), 
        axis=1
    )

    df['Hedge Entry Price'] = np.nan
    df['Hedge Exit Price'] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]['Week'])
        hedge_strike = df.iloc[i]['Hedge Strike']
        hedge_expiry = df.iloc[i]['Expiry']
        hedge_opt_type = df.iloc[i]['Contract'][-2:-1]
        hedge_entry_datetime = df.iloc[i]['Entry Time']
        hedge_exit_datetime = df.iloc[i]['Exit Time']

        hedge_df = await fetch_option_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if len(hedge_df) != 63:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]['c']
        hedge_exit_price = hedge_df.iloc[-1]['c']

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, 'Hedge Entry Price'] = hedge_entry_price
        df.loc[i, 'Hedge Exit Price'] = hedge_exit_price
        
    
    return df

tb_with_hedge = await add_hedges(tradebook)
tb_with_hedge

In [11]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [12]:
# tb_with_hedge['Hedge Points'] = tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
# tb_with_hedge['PnL'] = tb_with_hedge['Points'] * tb_with_hedge['Qty']
# tb_with_hedge['Hedge PnL'] = tb_with_hedge['Hedge Points'] * tb_with_hedge['Qty']
# tb_with_hedge['Total PnL'] = tb_with_hedge['PnL'] + tb_with_hedge['Hedge PnL']

In [13]:
# tb_with_hedge['ROI%'] = tb_with_hedge['Total PnL'] * 100 * 4 / 1000000
# tb_with_hedge['Entry Time'] = pd.to_datetime(tb_with_hedge['Entry Time'])
# tb_with_hedge['Trade Year'] = tb_with_hedge['Entry Time'].dt.year

In [28]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI%'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['ROI%'] > 0]['ROI%'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['ROI%'] < 0]['ROI%'].mean()
overall_max_drawdown = (combined_df_sorted['ROI%'].cumsum() - combined_df_sorted['ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,24.5635,63,57.1429,1.8890,-1.7376,-8.5518,2.8723
2018,33.6076,73,57.5342,2.2870,-2.2303,-11.3260,2.9673
2019,47.0510,75,61.3333,2.6191,-2.6224,-11.7143,4.0166
2020,197.6839,64,78.1250,4.6773,-2.7830,-13.7077,14.4214
2021,44.4423,74,56.7568,3.7295,-4.0071,-18.4486,2.4090
2022,73.7915,81,54.3210,3.5849,-2.5437,-10.6415,6.9343
2023,2.0864,75,49.3333,1.6726,-2.4916,-15.4864,0.1347
2024,12.8461,17,76.4706,1.9774,-3.2152,-8.0175,1.6023
Overall,436.0722,522.0000,59.3870,2.9689,-2.6464,-18.4486,23.6372


In [15]:
a = tb_with_hedge['PnL'].sum()
b = tb_with_hedge['Hedge PnL'].sum()
c = tb_with_hedge['Total PnL'].sum()
d = tb_with_hedge['ROI%'].sum()
print(a,b,c,d)

4360722.032340974 -3578995.261315308 787698.5356817511 315.07941427270043


In [29]:
tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [ ]:
tb_with_hedge

In [27]:
tb_with_hedge['ROI%'] = tb_with_hedge['PnL'] * 100 / 1000000